In [2]:
%pip install numpy pandas matploylib

Note: you may need to restart the kernel to use updated packages.


ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11; 1.26.0 Requires-Python >=3.9,<3.13; 1.26.1 Requires-Python >=3.9,<3.13
ERROR: Could not find a version that satisfies the requirement matploylib (from versions: none)

[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for matploylib


In [ ]:

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score



DATA_DIR = "E:\\SMCShiranth\\1999 Czech Financial Dataset"  # change if your CSVs are in a different folder

FOCUS_ACCOUNT_ID = 5215 

MONTHLY_BUDGET_CZK = 3200

# Output files
OUT_DIR = "outputs_czech1999"
os.makedirs(OUT_DIR, exist_ok=True)


In [ ]:
# Check what files are available in the directory
print("Files in current directory:")
print(os.listdir(DATA_DIR))

# If your CSV files are in a different location, update DATA_DIR here
# For example: DATA_DIR = "path/to/your/csv/files"

# Update DATA_DIR to the correct path where your CSV files are located
# DATA_DIR = "path/to/your/csv/files"  # Uncomment and set the correct path


# 1) LOAD TABLES 
account  = pd.read_csv(os.path.join(DATA_DIR, "account.csv"),  sep=";")
client   = pd.read_csv(os.path.join(DATA_DIR, "client.csv"),   sep=";")
disp     = pd.read_csv(os.path.join(DATA_DIR, "disp.csv"),     sep=";")
district = pd.read_csv(os.path.join(DATA_DIR, "district.csv"), sep=";")
card     = pd.read_csv(os.path.join(DATA_DIR, "card.csv"),     sep=";")
loan     = pd.read_csv(os.path.join(DATA_DIR, "loan.csv"),     sep=";")
order    = pd.read_csv(os.path.join(DATA_DIR, "order.csv"),    sep=";")

# trans.csv is huge → load only needed columns to save memory
trans_cols = ["trans_id", "account_id", "date", "type", "operation", "amount", "balance", "k_symbol", "bank", "account"]
trans = pd.read_csv(
    os.path.join(DATA_DIR, "trans.csv"),
    sep=";",
    usecols=trans_cols,
    low_memory=False
)

print("Loaded tables:")
print("account :", account.shape)
print("client  :", client.shape)
print("disp    :", disp.shape)
print("district:", district.shape)
print("card    :", card.shape)
print("loan    :", loan.shape)
print("order   :", order.shape)
print("trans   :", trans.shape)

Files in current directory:
['account.csv', 'card.csv', 'client.csv', 'data map.gif', 'disp.csv', 'district.csv', 'loan.csv', 'order.csv', 'trans.csv']
Loaded tables:
account : (4500, 4)
client  : (5369, 3)
disp    : (5369, 4)
district: (77, 16)
card    : (892, 4)
loan    : (682, 7)
order   : (6471, 6)
trans   : (1056320, 10)


In [ ]:
# -----------------------------
# 2) DATE PARSING (PKDD format = yymmdd as int)
# -----------------------------
def parse_pkdd_date(series):
    s = series.astype(str).str.zfill(6)
    return pd.to_datetime(s, format="%y%m%d", errors="coerce")

account["account_date"] = parse_pkdd_date(account["date"])
loan["loan_date"]       = parse_pkdd_date(loan["date"])
trans["trans_date"]     = parse_pkdd_date(trans["date"])
card["card_issued"]     = parse_pkdd_date(card["issued"])


# 3) BUILD RELATIONAL MASTER VIEWS

acc_disp_client = (
    account.merge(disp, on="account_id", how="left")
           .merge(client, on="client_id", how="left")
)


# 1) Find the district id column in client (or account_info view)
client_cols = set(acc_disp_client.columns)

# common possibilities in Czech1999 versions
possible_client_keys = ["district_id", "district", "A1", "district_code", "district_id_x"]
client_key = next((c for c in possible_client_keys if c in client_cols), None)

if client_key is None:
    raise KeyError(
        f"Couldn't find a district key in acc_disp_client. "
        f"Available columns include: {sorted(list(client_cols))[:40]} ..."
    )

# 2) Find the district id column in district table
district_cols = set(district.columns)
possible_district_keys = ["district_id", "A1", "district", "district_code"]
district_key = next((c for c in possible_district_keys if c in district_cols), None)

if district_key is None:
    raise KeyError(
        f"Couldn't find a district key in district.csv. "
        f"District columns are: {sorted(list(district_cols))}"
    )

# 3) Standardize district key names before merge
district_renamed = district.rename(columns={district_key: "district_join_key"})
acc_disp_client = acc_disp_client.rename(columns={client_key: "district_join_key"})

# 4) Merge
acc_disp_client = acc_disp_client.merge(district_renamed, on="district_join_key", how="left")

print("✅ District merge completed using:", client_key, "↔", district_key)
print("acc_disp_client shape:", acc_disp_client.shape)



✅ District merge completed using: district_id_x ↔ A1
acc_disp_client shape: (5369, 25)


In [8]:
# Add card info via disp_id
acc_disp_client = acc_disp_client.merge(card, on="disp_id", how="left")

print("\nacc_disp_client view after adding card info:", acc_disp_client.shape)
print("acc_disp_client columns:", list(acc_disp_client.columns))



acc_disp_client view after adding card info: (5369, 29)
acc_disp_client columns: ['account_id', 'district_join_key', 'frequency', 'date', 'account_date', 'disp_id', 'client_id', 'type_x', 'birth_number', 'district_id_y', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'card_id', 'type_y', 'issued', 'card_issued']


In [9]:
# -----------------------------
# 4) CHOOSE A FOCUS ACCOUNT
#    - If FOCUS_ACCOUNT_ID is None: pick the account with most transactions
# -----------------------------
if FOCUS_ACCOUNT_ID is None:
    top_acc = trans["account_id"].value_counts().head(1).index[0]
    FOCUS_ACCOUNT_ID = int(top_acc)

print(f"\nFOCUS_ACCOUNT_ID = {FOCUS_ACCOUNT_ID}")

# Filter dataset to focus account transactions
t = trans[trans["account_id"] == FOCUS_ACCOUNT_ID].copy()
t = t.dropna(subset=["trans_date", "amount"])

# Merge account details (optional)
account_info = acc_disp_client[acc_disp_client["account_id"] == FOCUS_ACCOUNT_ID].copy()

print("\nTransactions for focus account:", t.shape)
print("Account profile rows:", account_info.shape)


FOCUS_ACCOUNT_ID = 5215

Transactions for focus account: (637, 11)
Account profile rows: (1, 29)


In [ ]:

# 5) EDA + VISUALIZATIONS

t["flow"] = np.where(t["amount"] >= 0, "Income", "Expense")
t["expense_abs"] = t["amount"].abs()

total_income  = t.loc[t["flow"] == "Income", "amount"].sum()
total_expense = t.loc[t["flow"] == "Expense", "expense_abs"].sum()
net           = total_income - total_expense

print("\n--- BASIC FINANCIAL SUMMARY ---")
print(f"Total Income : {total_income:,.2f} CZK")
print(f"Total Expense: {total_expense:,.2f} CZK")
print(f"Net          : {net:,.2f} CZK")

# 5.2 Monthly aggregation
t["year"]  = t["trans_date"].dt.year
t["month"] = t["trans_date"].dt.month
t["ym"]    = t["trans_date"].dt.to_period("M").astype(str)

monthly = (
    t.groupby("ym")
     .agg(
        total_spending=("expense_abs", "sum"),
        transaction_count=("amount", "count"),
        avg_transaction=("expense_abs", "mean")
     )
     .reset_index()
)

monthly["month_index"] = np.arange(1, len(monthly) + 1)

monthly_path = os.path.join(OUT_DIR, "monthly_spending.csv")
monthly.to_csv(monthly_path, index=False)
print(f"\nSaved monthly table → {monthly_path}")

# 5.3 Chart: Monthly spending trend
plt.figure()
plt.plot(monthly["month_index"], monthly["total_spending"], marker="o")
plt.title(f"Monthly Spending Trend (Account {FOCUS_ACCOUNT_ID})")
plt.xlabel("Month Index")
plt.ylabel("Total Spending (CZK)")
plt.grid(True)
trend_path = os.path.join(OUT_DIR, "monthly_trend.png")
plt.savefig(trend_path, dpi=200, bbox_inches="tight")
plt.close()
print(f"Saved chart → {trend_path}")

# 5.4 Category breakdown (k_symbol)
# Some rows have missing/blank k_symbol
t["k_symbol"] = t["k_symbol"].fillna("UNKNOWN").astype(str).str.strip()
expense_rows = t[t["flow"] == "Expense"].copy()

cat_spend = expense_rows.groupby("k_symbol")["expense_abs"].sum().sort_values(ascending=False).head(12)

plt.figure()
plt.barh(cat_spend.index[::-1], cat_spend.values[::-1])
plt.title("Top Spending Categories (k_symbol)")
plt.xlabel("Total Spending (CZK)")
plt.ylabel("Category")
cat_path = os.path.join(OUT_DIR, "category_bar.png")
plt.savefig(cat_path, dpi=200, bbox_inches="tight")
plt.close()
print(f"Saved chart → {cat_path}")

# 5.5 Day-of-month spending distribution
expense_rows["day"] = expense_rows["trans_date"].dt.day
daily = expense_rows.groupby("day")["expense_abs"].sum().reindex(range(1, 32), fill_value=0)

plt.figure()
plt.bar(daily.index, daily.values)
plt.title("Total Spending by Day of Month")
plt.xlabel("Day of Month")
plt.ylabel("Total Spending (CZK)")
day_path = os.path.join(OUT_DIR, "spending_by_day.png")
plt.savefig(day_path, dpi=200, bbox_inches="tight")
plt.close()
print(f"Saved chart → {day_path}")



--- BASIC FINANCIAL SUMMARY ---
Total Income : 2,226,418.50 CZK
Total Expense: 0.00 CZK
Net          : 2,226,418.50 CZK

Saved monthly table → outputs_czech1999\monthly_spending.csv
Saved chart → outputs_czech1999\monthly_trend.png
Saved chart → outputs_czech1999\category_bar.png
Saved chart → outputs_czech1999\spending_by_day.png


In [ ]:

# 6)ANOMALY DETECTION (IQR)
x = expense_rows["expense_abs"]
q1, q3 = x.quantile(0.25), x.quantile(0.75)
iqr = q3 - q1
upper = q3 + 1.5 * iqr

anomalies = expense_rows[expense_rows["expense_abs"] > upper].copy()
anomalies = anomalies.sort_values("expense_abs", ascending=False)

anom_path = os.path.join(OUT_DIR, "anomalies_top.csv")
anomalies[["trans_date", "expense_abs", "k_symbol", "operation", "balance"]].head(50).to_csv(anom_path, index=False)

print("\n--- ANOMALY DETECTION (IQR) ---")
print(f"IQR Upper Threshold: {upper:,.2f} CZK")
print(f"Anomalies Found    : {len(anomalies)}")
print(f"Saved anomalies → {anom_path}")

print("\nTop 5 anomalies:")
print(anomalies[["trans_date", "expense_abs", "k_symbol", "operation"]].head(5))



--- ANOMALY DETECTION (IQR) ---
IQR Upper Threshold: nan CZK
Anomalies Found    : 0
Saved anomalies → outputs_czech1999\anomalies_top.csv

Top 5 anomalies:
Empty DataFrame
Columns: [trans_date, expense_abs, k_symbol, operation]
Index: []


In [ ]:

# 7)AI MODEL (Linear Regression Spending Forecast)
X = monthly[["month_index"]].values
y = monthly["total_spending"].values

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

r2   = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae  = mean_absolute_error(y_test, y_pred)

cv_r2 = cross_val_score(model, X, y, cv=5, scoring="r2")

print("\n--- LINEAR REGRESSION MODEL ---")
print(f"Equation: spending = {model.coef_[0]:.2f} * month_index + {model.intercept_:.2f}")
print(f"R² (test): {r2:.4f}")
print(f"RMSE     : {rmse:,.2f} CZK")
print(f"MAE      : {mae:,.2f} CZK")
print(f"CV R²    : {cv_r2.mean():.4f} (+/- {cv_r2.std():.4f})")

# Forecast next 6 months
last_idx = int(monthly["month_index"].max())
future_idx = np.arange(last_idx + 1, last_idx + 7).reshape(-1, 1)
future_pred = model.predict(future_idx)


--- LINEAR REGRESSION MODEL ---
Equation: spending = 18.80 * month_index + 32914.15
R² (test): 0.0156
RMSE     : 7,646.06 CZK
MAE      : 6,211.74 CZK
CV R²    : -0.0460 (+/- 0.0296)


In [ ]:

# 8) DSS RISK ZONES + ALERTS
def risk_zone(pred, budget):
    if pred > budget:
        return "HIGH"
    elif pred > 0.80 * budget:
        return "MEDIUM"
    return "LOW"

forecast_df = pd.DataFrame({
    "month_index": future_idx.flatten(),
    "predicted_spending_czk": future_pred,
})
forecast_df["risk"] = forecast_df["predicted_spending_czk"].apply(lambda v: risk_zone(v, MONTHLY_BUDGET_CZK))

forecast_path = os.path.join(OUT_DIR, "forecast_6_months.csv")
forecast_df.to_csv(forecast_path, index=False)
print(f"\nSaved forecast → {forecast_path}")

# Plot forecast + risk bands
plt.figure()
plt.scatter(monthly["month_index"], monthly["total_spending"], label="Actual Spending")
plt.plot(monthly["month_index"], model.predict(X), label="Trend (Linear Regression)")
plt.scatter(forecast_df["month_index"], forecast_df["predicted_spending_czk"], label="Forecast (6 months)")

plt.axhspan(0, 0.80 * MONTHLY_BUDGET_CZK, alpha=0.2, label="LOW (<80% budget)")
plt.axhspan(0.80 * MONTHLY_BUDGET_CZK, MONTHLY_BUDGET_CZK, alpha=0.2, label="MED (80–100%)")

max_y = max(monthly["total_spending"].max(), forecast_df["predicted_spending_czk"].max()) * 1.10
plt.axhspan(MONTHLY_BUDGET_CZK, max_y, alpha=0.2, label="HIGH (>budget)")

plt.title("Spending Forecast with DSS Risk Zones")
plt.xlabel("Month Index")
plt.ylabel("Spending (CZK)")
plt.legend()
risk_plot_path = os.path.join(OUT_DIR, "forecast_risk_zones.png")
plt.savefig(risk_plot_path, dpi=200, bbox_inches="tight")
plt.close()
print(f"Saved chart → {risk_plot_path}")

# Audio-ready DSS output strings (for accessibility)
print("\n--- DSS AUDIO-READY ALERTS ---")
for _, row in forecast_df.iterrows():
    mi = int(row["month_index"])
    pred = row["predicted_spending_czk"]
    risk = row["risk"]

    msg = f"Month {mi}: predicted spending is {pred:,.0f} CZK. Risk level is {risk}."
    if risk == "HIGH":
        msg += f" Warning: this exceeds your budget of {MONTHLY_BUDGET_CZK:,.0f} CZK."
    elif risk == "MEDIUM":
        msg += " Caution: you are close to your budget."
    else:
        msg += " You are within a safe spending range."
    print(msg)



Saved forecast → outputs_czech1999\forecast_6_months.csv
Saved chart → outputs_czech1999\forecast_risk_zones.png

--- DSS AUDIO-READY ALERTS ---
Month 67: predicted spending is 34,174 CZK. Risk level is HIGH. Warning: this exceeds your budget of 3,200 CZK.
Month 68: predicted spending is 34,192 CZK. Risk level is HIGH. Warning: this exceeds your budget of 3,200 CZK.
Month 69: predicted spending is 34,211 CZK. Risk level is HIGH. Warning: this exceeds your budget of 3,200 CZK.
Month 70: predicted spending is 34,230 CZK. Risk level is HIGH. Warning: this exceeds your budget of 3,200 CZK.
Month 71: predicted spending is 34,249 CZK. Risk level is HIGH. Warning: this exceeds your budget of 3,200 CZK.
Month 72: predicted spending is 34,268 CZK. Risk level is HIGH. Warning: this exceeds your budget of 3,200 CZK.


In [ ]:
print("\n--- ACCOUNT CONTEXT (first row) ---")
if len(account_info) > 0:
    cols_to_show = ["account_id", "district_id", "frequency", "account_date", "type_x", "client_id", "birth_number", "A2"]
    # type_x is disp.type after merges, A2 is district name (in many versions)
    existing = [c for c in cols_to_show if c in account_info.columns]
    print(account_info[existing].head(1).to_string(index=False))
else:
    print("No account profile found (check if account_id exists in disp/client joins).")

print(f"\nDONE ✅ Outputs saved in: {OUT_DIR}")


--- ACCOUNT CONTEXT (first row) ---
 account_id        frequency account_date type_x  client_id  birth_number            A2
       5215 POPLATEK MESICNE   1993-07-01  OWNER       6301        495308 Brno - venkov

DONE ✅ Outputs saved in: outputs_czech1999
